# Assessment of Secondary Structure Prediction
**Author:** Charles Hoyt, LSI, B-IT, University of Bonn

**Goal:** Assess the quality of [`PSIPRED`](http://bioinf.cs.ucl.ac.uk/psipred/) and [`JPred`](http://www.compbio.dundee.ac.uk/jpred/) in comparison to secondary structure calculations from [MOE](https://www.chemcomp.com/MOE-Molecular_Operating_Environment.htm) on PDB Structure [1XP0](www.pdb.org/pdb/explore/explore.do?structureId=1XP0) . 

## Raw Data
The data from MOE was calculated and transcribed (by hand). `PSIPRED` and `JPred` were both used to predict the secondary structure of 1XP0 given its FASTA sequence.

In [1]:
moe     = """--HHHHHHHHHHH-----HHHHTTT-TT---TT--HHHHHHHHHHHHHHTTHHHHTT--HHHHHHHHHHHHHT--TT-----HHHHHHHHHHHHHHHHTT--HHH--HHHHHHHHHHHHHTTTT-----HHHHHHTT-HHHHHTTT--HHHHHHHHHHHHHHT-TT--TTTT--HHHHHHHHHHHHHHHHHT-HHHHHHHHHHHHHHHHTT---TT-HHHHHHHHHHHHHHHHTHHHH--HHHHHHHHHHHHHHHHHHHHHHHHHT-----HHH-HHHHHHHHHHHHHHHHHTTHHHHHHHHHH-HHHHHHHHHHHHHHHHHHHHT--"""
psipred = """----HHHHHHHHH-----------------------HHHHHHHHHHHHHH---------HHHHHHHHHHH-------------HHHH-HHHHHHHHHH---HHHHHHHHHHHHHHHHHHHH---------HHHHH---HHHHH-----HHHHHHHHHHHHHH------------HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH---------HHHHHHHHHHHHHH--------HHHHHHHHHHHHHH----HHHHHH---------------------HHHHHHHHHHHHHHHHHH----HHHHHHHHHHHHHHHHHHHH"""
jpred   = """-E--HHHHHHH-------HHH----------------HHHHHHHHHHHH----------HHHHHHHHHHHHHH---------HHHHHHHHHHHHHHHHH---------HHHHHHHHHHHHH---------HHHHHH---HHHH-----HHHHHHHHHHHHHH------------HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH-----------HHHHHHHHHHH---------HHHHHHHHHHHHHHHHHHHHHHHHH-------------------HHHHHHHHHHHHHHHHHHHH---HHHHHHHHHHHHHHHHHHHHH""" 

## Preprocessing of Data
Sensitivity, Specificity, F1, and Matthew's Correlation Coefficient are all measured applied to binary classifications, they must be modified to fit the ternary classification of protein secondary structure. In this analysis, the presence of a secondary structure (alpha helix or beta sheet) is considered a positive result, while the absence of secondary structure (a random coil) is considered a negative result.

Additionally, MOE calculates turns and denotes them as `T`. In this analysis, these are taken as random coils.

In [2]:
jpred = jpred.replace("E", "H")

moe = moe.replace("T", "-")

## Tabulation of [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix)
The confusion matrix tabulates the discrepencies between the actual result (from MOE) and the predicted results.

It can also be calculated using [SciKit Learn](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) and more advanced statistics can be calculated on confusion matricies larger than 2x2 with SciKit's [Classification Report](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-report)


In [3]:
p_fn, p_fp, p_tp, p_tn = 0, 0, 0, 0
j_fn, j_fp, j_tp, j_tn = 0, 0, 0, 0

for m, p, j in zip(moe, psipred, jpred):
    # Statistics over the PSIPRED Data
    if m == 'H' and p == '-':
        p_fn += 1
    if m == '-' and p == 'H':
        p_fp += 1
    if m == 'H' and p == 'H':
        p_tp += 1
    if m == '-' and p == '-':
        p_tn += 1   
    
    # Statistics over the JPred Data
    if m == 'H' and j == '-':
        j_fn += 1
    if m == '-' and j == 'H':
        j_fp += 1
    if m == 'H' and j == 'H':
        j_tp += 1
    if m == '-' and j == '-':
        j_tn += 1
        
print("PSIPRED tp {} tn {} fp {} fn {}".format(p_tp, p_tn, p_fp, p_fn))    
print("JPred   tp {} tn {} fp {} fn {}".format(j_tp, j_tn, j_fp, j_fn))

PSIPRED tp 187 tn 84 fp 13 fn 44
JPred   tp 191 tn 85 fp 12 fn 40


## Sensitivity

Sensitivity measures the proportion of correctly identified positive values. In this example, positives are helicies. It is calculated by:

$Sensitivity=\dfrac{TP}{P}=\dfrac{TP}{TP+FN}$

In [4]:
p_sens = p_tp / (p_tp + p_fn)
j_sens = j_tp / (j_tp + j_fn)

print("Method\tSensitivity\nPSIPRED\t{:.2%}\nJPred\t{:.2%}".format(p_sens,j_sens))

Method	Sensitivity
PSIPRED	80.95%
JPred	82.68%


## Specificity
Specifity measures the proportion of correctly identified negative values. In this example, negatives are coils. It is calculated by:

$Specificity=\dfrac{TN}{N}=\dfrac{TN}{TN+FP}$

In [5]:
p_spec = p_tn / (p_tn + p_fp)
j_spec = j_tn / (j_tn + j_fp)

print("Method\tSpecificity\nPSIPRED\t{:.2%}\nJPred\t{:.2%}".format(p_spec,j_spec))

Method	Specificity
PSIPRED	86.60%
JPred	87.63%


## F1 Score
F1 Score is the harmonic mean of sensitivity and specificity. It is a good "average" that can be reported easily. It is calculated by:

$F1=\dfrac{2*TP}{2*TP+FP+FN}$

In [6]:
p_f1 = 2 * p_tp / (2 * p_tp + p_fp + p_fn)
j_f1 = 2 * j_tp / (2 * j_tp + j_fp + j_fn)

print("Method\tF1\nPSIPRED\t{:.2%}\nJPred\t{:.2%}".format(p_f1, j_f1))

Method	F1
PSIPRED	86.77%
JPred	88.02%


## Matthew's Correlation Coefficient
The Matthew's Correlation Coefficient additionally takes both TP and TN into account. It is calculated by:

$MCC=\dfrac{TP*TN-FP*FN}{\sqrt{(TP+FP)*(TP+FN)*(TN+FN)*(TN+FN)}}$

In [7]:
from math import sqrt
p_mcc = (p_tp * p_tn - p_fp * p_fn) / sqrt((p_tp + p_fp) * (p_tp + p_fn) * (p_tn + p_fn) * (p_tn + p_fn))
j_mcc = (j_tp * j_tn - j_fp * j_fn) / sqrt((j_tp + j_fp) * (j_tp + j_fn) * (j_tn + j_fn) * (j_tn + j_fn))

print("Method\tMCC\nPSIPRED\t{:.2%}\nJPred\t{:.2%}".format(p_mcc, j_mcc))

Method	MCC
PSIPRED	55.01%
JPred	58.20%


## Conclusions

Each of the sensitivity, specificity, F1, and Matthew's Correlation coefficient of the `JPred` methods were better for PDB 1XP0. Both `PSIPRED` and `JPred` were successful at identifying regions of helicies, but were often erroneous in their lengths.

This analysis represents a simple case - for other proteins with folds that are not all-alpha, this type of analysis will be slightly more complicated. These prediction methods may also have different performance on proteins of different folds. Ultimately, the results from comparisons to 1XP0 can be contextualized with the results of the WS1516 Structural Bioinformatics class on a wide variety of proteins.

## References
* https://en.wikipedia.org/wiki/Sensitivity_and_specificity